In [1]:
import psycopg2
from age.networkx import *
from age import *
import networkx as nx

# conn = psycopg2.connect(
#     host="localhost",
#     port="5430",
#     dbname="postgresDB",
#     user="postgresUser",
#     password="postgresPW")
# graphName = 'bitnine_global_inic'

conn = psycopg2.connect(
    host="localhost",
    port="5432",
    dbname="postgres",
    user="moontasir",
    password="254826")
graphName = 'bitnine_global_inic'

# Making sure we are working with a new graph
age.setUpAge(conn, graphName)
age.deleteGraph(conn,graphName)
age.setUpAge(conn, graphName)

### Creating a Random Networkx Graph

In [2]:
import random
num_nodes = 5
num_edges = 10
G = nx.DiGraph()
try:
    for i in range(num_nodes//2):
        G.add_node(i, label='Number',properties={'name' : i})
    for i in range(num_nodes//2,num_nodes):
        G.add_node(i, label='Integer',properties={'age' : i*2} )
    for i in range(num_edges):
        source = random.randint(0, num_nodes-1)
        target = random.randint(0, num_nodes-1)
        G.add_edge(source, target, label='Connection' ,properties={'st' : source , 'ed':target})
except Exception as e:
    raise e
print(G)

DiGraph with 5 nodes and 9 edges


## Networkx to AGE

In [3]:
networkx_to_age(conn, G, graphName)

## AGE to Networkx

### ALL AGE to Networkx


In [4]:
G = age_to_networkx(conn, graphName)
print(G)

DiGraph with 5 nodes and 9 edges


### Add subgraph of AGE to Networkx using query

In [5]:
G = age_to_networkx(conn, graphName, 
                    query="""
SELECT * from cypher('%s', $$
        MATCH (V:Integer)
        RETURN V
$$) as (V agtype);
""" % graphName)

G = age_to_networkx(conn, graphName, G=G,
                    query="""
SELECT * from cypher('%s', $$
        MATCH (V:Number)
        RETURN V
$$) as (V agtype);
""" % graphName)

G = age_to_networkx(conn, graphName, G=G,
                    query="""
SELECT * from cypher('%s', $$
        MATCH (V)-[R]->(V2)
        RETURN V,R,V2
$$) as (V agtype, R agtype, V2 agtype);
""" % graphName)
print(G)


DiGraph with 5 nodes and 9 edges


## Loading `AGE load()` data into AGE

In [6]:
age.deleteGraph(conn,graphName)
age.setUpAge(conn, graphName)
with conn.cursor() as cursor:
    cursor.execute(
        """
        SELECT create_vlabel('bitnine_global_inic','Country');
        SELECT create_vlabel('bitnine_global_inic','City');
        SELECT create_elabel('bitnine_global_inic','has_city');
        
        SELECT load_labels_from_file('bitnine_global_inic', 'Country','/home/moontasir/Desktop/temp_age/age/regress/age_load/data/countries.csv');

        SELECT load_labels_from_file('bitnine_global_inic', 'City','/home/moontasir/Desktop/temp_age/age/regress/age_load/data/cities.csv');

        SELECT load_edges_from_file('bitnine_global_inic', 'has_city','/home/moontasir/Desktop/temp_age/age/regress/age_load/data/edges.csv');
        """)
    conn.commit()
    for row in cursor:
        print(row)

with conn.cursor() as cursor:
    cursor.execute(
        """SELECT COUNT(*) from cypher('%s', $$
            MATCH (V)
            RETURN V
        $$) as (V agtype);""" % graphName)
    for row in cursor:
        print(row)

with conn.cursor() as cursor:
    cursor.execute(
        """SELECT COUNT(*) from cypher('%s', $$
        MATCH (V)-[R]->(V2)
        RETURN V,R,V2
$$) as (V agtype, R agtype, V2 agtype);""" % graphName)
    for row in cursor:
        print(row)

('',)
(72538,)
(72485,)


In [7]:
G = age_to_networkx(conn, graphName)
print(G)

DiGraph with 72538 nodes and 72485 edges


In [8]:
G2 = age_to_networkx(conn,graphName, query="""
SELECT * from cypher('%s', $$
        MATCH (V)-[R]->(V2)
        RETURN V,R,V2
$$) as (V agtype, R agtype, V2 agtype);
""" % graphName)
print(G2)

DiGraph with 72528 nodes and 72485 edges


In [9]:
age.deleteGraph(conn,graphName)
age.setUpAge(conn, graphName)

In [10]:
networkx_to_age(conn, G, graphName)

In [11]:
age.deleteGraph(conn,graphName)
age.setUpAge(conn, graphName)